In [ ]:
import tensorflow as tf
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

In [ ]:
def get_tensorboard_log_as_dataframe(logdir):
    data = []
    exclude_pattern = re.compile(r'bayesian_expected_sarsa__\d+')
    for subdir, _, files in os.walk(logdir):
        if exclude_pattern.search(os.path.basename(subdir)):
            continue
        for file in files:
            if "events.out.tfevents" in file:
                event_file = os.path.join(subdir, file)
                for event in tf.compat.v1.train.summary_iterator(event_file):
                    for value in event.summary.value:
                        if value.HasField('simple_value'):
                            data.append({
                                'event':subdir[5:],
                                'step': event.step,
                                'tag': value.tag,
                                'value': value.simple_value
                            })

    df = pd.DataFrame(data)
    df.sort_values("step")
    return df

logdir = "runs"
df = get_tensorboard_log_as_dataframe(logdir)

In [ ]:
envs = ["BreakoutNoFrameskip", "Freeway","SpaceInvaders"]
colors = ["c","r","y","m"]
for env in envs:
    env_ep_ret = df[df.event.str.contains(env) & df.tag.str.contains("charts/episodic_return")]
    plt.rcParams.update({"font.size": 22})
    plt.figure(figsize=(15, 6))
    i = 0
    for event in env_ep_ret.event.drop_duplicates().sort_values():
        data = env_ep_ret[env_ep_ret.event == event]
        plt.plot(data.step,data.value, label=event.split("__",1)[0], color=colors[i])
        i+=1

    plt.title(f"Episode Return in {env}")
    plt.ylabel("Episode Return")
    plt.xlabel("Episode")
    plt.grid(axis = 'y')
    plt.legend()
    plt.savefig(f'imgs/{env}_ep_return.png')

    env_ep_mean_ret = df[df.event.str.contains(env) & df.tag.str.contains("eval/mean_episodic_return")]
    plt.rcParams.update({"font.size": 22})
    plt.figure(figsize=(15, 6))
    i = 0
    for event in env_ep_mean_ret.event.drop_duplicates().sort_values():
        data = env_ep_mean_ret[env_ep_mean_ret.event == event]
        plt.plot(data.step,data.value, label=event.split("__",1)[0], color=colors[i])
        i+=1

    plt.title(f"Mean Episode Return in {env}")
    plt.ylabel("Episode Return")
    plt.xlabel("Episode")
    plt.grid(axis = 'y')
    plt.legend()
    plt.savefig(f'imgs/{env}_ep_mean_return.png')

In [ ]:
def get_tensorboard_log_as_dataframe(logdir):
    data = []
    include_pattern = re.compile(r'bayesian_expected_sarsa__\d+')
    for subdir, _, files in os.walk(logdir):
        if include_pattern.search(os.path.basename(subdir)):
            for file in files:
                if "events.out.tfevents" in file:
                    event_file = os.path.join(subdir, file)
                    for event in tf.compat.v1.train.summary_iterator(event_file):
                        for value in event.summary.value:
                            if value.HasField('simple_value'):
                                data.append({
                                    'event':subdir[5:],
                                    'step': event.step,
                                    'tag': value.tag,
                                    'value': value.simple_value
                                })

    df = pd.DataFrame(data)
    df.sort_values("step")
    return df

logdir = "bayesian_exp"
df = get_tensorboard_log_as_dataframe(logdir)

In [ ]:
colors = ["c","r","y","m", "b", "g"]

env_ep_ret = df[df.tag.str.contains("charts/episodic_return")]
plt.rcParams.update({"font.size": 22})
plt.figure(figsize=(15, 6))
i = 0
for event in env_ep_ret.event.drop_duplicates().sort_values():
    freq = event.split("__",2)[1]
    data = env_ep_ret[env_ep_ret.event == event]
    plt.plot(data.step,data.value, label=f"{freq}", color=colors[i])
    i+=1

plt.title(f"Episode Return For Different Update Intervals")
plt.ylabel("Episode Return")
plt.xlabel("Episode")
plt.grid(axis = 'y')
plt.legend()
plt.savefig(f'imgs/Frequencies_ep_return.png')

env_ep_mean_ret = df[df.tag.str.contains("eval/mean_episodic_return")]
plt.rcParams.update({"font.size": 22})
plt.figure(figsize=(15, 6))
i = 0
for event in env_ep_mean_ret.event.drop_duplicates().sort_values():
    freq = event.split("__",2)[1]
    data = env_ep_mean_ret[env_ep_mean_ret.event == event]
    plt.plot(data.step,data.value, label=f"{freq}", color=colors[i])
    i+=1

plt.title(f"Mean Episode Return For Different Update Intervals")
plt.ylabel("Episode Return")
plt.xlabel("Episode")
plt.grid(axis = 'y')
plt.legend()
plt.savefig(f'imgs/Frequencies_mean_ep_return.png')
